## Making the trianable ansatz

In [44]:
from pennylane import numpy as np
import pennylane as qml
class Ansatz():
    def __init__(self, num_layers=30, seed=np.random.randint(10, 10000)):
        np.random.seed(seed)
        self.num_layers = num_layers
        self.n_qubits = 6
        self.dev = qml.device('default.qubit', wires=self.n_qubits)
        self.weights = 2 * np.random.random(size=(self.num_layers, 5), requires_grad=True) - 1 # range from -1 to 1
        self.defineStates() # makes sure we have all 5 basis states for logical CNOT
        # functions
        self.amp_sqrd = lambda c : np.real(c*np.conjugate(c))
        self.Udot = lambda s1, U, s2 : np.dot(np.conjugate(np.transpose(s1)),np.matmul(U,s2))

    def U_ex(self, p):
        from scipy.linalg import expm
        import numpy as npy
        X = [[0,1],[1,0]]
        Y = npy.array([[0,-1j],[1j,0]], dtype=npy.complex128)
        Z = [[1,0],[0,-1]]

        H_ex = (1/4)*(npy.kron(X,X) + npy.kron(Y,Y) + npy.kron(Z,Z))
        # print(f'H_ex.type = {type(H_ex)}')
        U_exchange = expm(-1j*npy.pi*p*H_ex)
        return np.array(U_exchange, requires_grad=False)

    def W(self, layer_weights):
        """Trainable circuit block."""
        # print(f'layer_weights.shape = {layer_weights.shape}, layer_weights[0] = {layer_weights[0]}')
        # print(f'binary val: {type(layer_weights[0]) == np.numpy_boxes.ArrayBox}')
        qml.QubitUnitary(self.U_ex(layer_weights[0]), wires=[1,2])
        qml.QubitUnitary(self.U_ex(layer_weights[1]), wires=[3,4])
        #
        qml.QubitUnitary(self.U_ex(layer_weights[2]), wires=[0,1])
        qml.QubitUnitary(self.U_ex(layer_weights[3]), wires=[2,3])
        qml.QubitUnitary(self.U_ex(layer_weights[4]), wires=[4,5])

    def Unitary_CNOT(self, weights):
        # print(f'weights.shape={weights.shape}')
        for layer_weights in weights:
            self.W(qml.math.toarray(layer_weights))

    def quantum_model(self, weights, inputStateVector):
        qml.AmplitudeEmbedding(inputStateVector, wires=range(self.n_qubits))
        self.Unitary_CNOT(weights)
        return qml.state()

    def get_predictions(self, weights):
        qnode_ = qml.QNode(self.quantum_model, self.dev, interface='autograd')
        vals = []
        for i in range(len(self.inputStates)):
            outputState = qnode_(weights, self.inputStates[i])
            c = np.dot(np.conjugate(self.expectedStates[i]), outputState)
            vals.append(self.amp_sqrd(c))
        return np.array(vals, requires_grad=False)
    
    def loss_function(self, weights):
        # qnode_U = qml.QNode(self.Unitary_CNOT, self.dev, interface='autograd')
        # U_cnot_prime = qml.matrix(self.Unitary_CNOT)(weights)
        values = self.get_predictions(weights)
        return np.sqrt(1 - (1/4)*sum(values))
    
    def cost(self, weights):
        return self.loss_function(weights)
    
    def train(self, max_steps=80, alpha=0.1):
        opt = qml.AdamOptimizer(alpha)

        cst = [self.cost(self.weights)]  # initial cost

        for step in range(max_steps):

            # Update the weights by one optimizer step
            temp = opt.step(self.cost, self.weights)
            # print(temp)
            # print(temp.shape)
            self.weights = temp

            # Save current cost
            c = self.cost(self.weights)
            cst.append(c)
            # if (step + 1) % 10 == 0:
            print("Cost at step {0:3}: {1}".format(step + 1, c))
    
    def defineStates(self):
        import numpy as np
        # used for the basis states
        def nestedKronecker(args): # use "*args" to access an array of inputs
            assert len(args) >= 2
            temp = args[0]
            for arg in args[1:]:
                temp = np.kron(temp, arg)
            return temp

        basis = {0: [1,0], 1: [0,1], '0': [1,0], '1': [0,1]}

        basisVector = lambda binstr : nestedKronecker([basis[x] for x in binstr])

        # common states
        zero, one = basis['0'], basis['1']
        tplus = basisVector('11')
        tminus = basisVector('00')
        tzero = (1/np.sqrt(2))*(basisVector('01') + basisVector('10'))
        singlet = np.sqrt(1/2)*(basisVector('01') - basisVector('10'))


        # ------------------------ FOR STATE 1 ------------------------

        state1 = np.kron(np.kron(singlet, singlet), singlet)

        # ------------------------ FOR STATE 2 ------------------------

        largePyramid = np.sqrt(1/3)*(np.kron(tplus,tminus)+np.kron(tminus,tplus)-np.kron(tzero,tzero))
        state2 = np.kron(singlet,largePyramid)

        # ------------------------ FOR STATE 3 ------------------------

        state3 = np.kron(largePyramid,singlet)

        # ------------------------ FOR STATE 4 ------------------------

        # for psi0 and psi1 we are combining j1=1 and j2=1/2 (this is combinind the first peak and trough)
        # J = 1/2, M = -1/2
        psi0 = np.sqrt(1/3)*np.kron(tzero, zero) - np.sqrt(2/3)*np.kron(tminus, one)
        # J = 1/2, M = +1/2
        psi1 = np.sqrt(2/3)*np.kron(tplus, zero) - np.sqrt(1/3)*np.kron(tzero, one)


        # for phiminus, phizero, phiplus, we are are combining j1=1/2 and j2=1/2
        # J = 1, M = -1
        phiminus = np.kron(psi0,zero)
        # J = 1, M = 0
        phizero = np.sqrt(1/2)*(np.kron(psi1,zero) + np.kron(psi0,one))
        # J = 1, M = +1
        phiplus = np.kron(psi1,one)

        # J=0,M=0 and j1=1,j2=1
        state4 = np.sqrt(1/3)*(np.kron(phiplus, tminus) - np.kron(phizero, tzero) + np.kron(phiminus, tplus))

        # ------------------------ FOR STATE 5 ------------------------

        eta_minus3 = np.kron(tminus, basis['0'])
        eta_minus1 = np.sqrt(2/3)*np.kron(tzero,zero) + np.sqrt(1/3)*np.kron(tminus,one)
        eta_plus1 = np.sqrt(1/3)*np.kron(tplus,zero) + np.sqrt(2/3)*np.kron(tzero, one)
        eta_plus3 = np.kron(tplus,one)

        gamma_minus = np.sqrt(1/4)*np.kron(eta_minus1, zero) - np.sqrt(3/4)*np.kron(eta_minus3, one)
        gamma_zero = np.sqrt(1/2)*np.kron(eta_plus1, zero) - np.sqrt(1/2)*np.kron(eta_minus1,one)
        gamma_plus = np.sqrt(3/4)*np.kron(eta_plus3, zero) - np.sqrt(1/4)*np.kron(eta_plus1, one)

        state5 = np.sqrt(1/3)*(np.kron(gamma_plus,tminus) - np.kron(gamma_zero, tzero) - np.kron(gamma_minus, tplus))

        self.inputStates = [state1, state2, state3, state4]
        self.expectedStates = [state1, state2, state4, state3]


In [45]:
fixedAnsatz = Ansatz(num_layers=25,seed=34324)
fixedAnsatz.train(max_steps=20, alpha=0.03)

/Users/danielmolina/opt/anaconda3/envs/qrl/lib/python3.10/site-packages/autograd/tracer.py:14: UserWarning: Output seems independent of input.
  warnings.warn("Output seems independent of input.")


Cost at step   1: 0.9488629732470668
Cost at step   2: 0.9488629732470668
Cost at step   3: 0.9488629732470668
Cost at step   4: 0.9488629732470668
Cost at step   5: 0.9488629732470668
Cost at step   6: 0.9488629732470668
Cost at step   7: 0.9488629732470668
Cost at step   8: 0.9488629732470668
Cost at step   9: 0.9488629732470668
Cost at step  10: 0.9488629732470668
Cost at step  11: 0.9488629732470668
Cost at step  12: 0.9488629732470668
Cost at step  13: 0.9488629732470668
Cost at step  14: 0.9488629732470668
Cost at step  15: 0.9488629732470668
Cost at step  16: 0.9488629732470668
Cost at step  17: 0.9488629732470668
Cost at step  18: 0.9488629732470668
Cost at step  19: 0.9488629732470668
Cost at step  20: 0.9488629732470668
